In [2]:
# Importing necessary libraries
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
from spotipy import Spotify

In [3]:
#Authenticating my account via Spotify WebAPI
client_credentials_mgmt = SpotifyClientCredentials(client_id= "2582d06265ec4d4fbdfd8c26e00fc794", client_secret= "24e812cea0024f3985f204d559156199")
sp = spotipy.Spotify(client_credentials_manager = client_credentials_mgmt)

In [4]:
# Importing my "Liked Songs" playlist
playlist_link = "https://open.spotify.com/playlist/078Ro1JBhOUr2TtTd8wgwz?si=7e31048995be4b36"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]

# Collecting each individual track URI
track_uris = []

for x in sp.playlist_tracks(playlist_URI)["items"]:
    track_uris.append(x["track"]["uri"])

In [5]:
# Initialize variables
track_data = []
playlist_id = "078Ro1JBhOUr2TtTd8wgwz"
results = sp.playlist_tracks(playlist_id, limit=100, offset=0)

#Creating DataFrame with each track's information
while results:
    for item in results["items"]:
        track = item["track"]
        track_info = {
            "track_uri": track["uri"],
            "track_name": track["name"],
            "artist_name": ", ".join(artist["name"] for artist in track["artists"]),
            "album_name": track["album"]["name"],
            "track_duration": track["duration_ms"]
        }
        track_data.append(track_info)
    
    # Checking if there are more tracks to retrieve
    if results["next"]:
        results = sp.next(results)
    else:
        results = None

# Creating and displaying a DataFrame
df = pd.DataFrame(track_data)
df

,track_uri,track_name,artist_name,album_name,track_duration
0,spotify:track:1p5jgCAGFDBIBs6ZbWIMvA,Bad Boy,Cascada,Everytime We Touch (Premium Edition),191843
1,spotify:track:3BhyxulZ13uWZS6LqiYiCj,Cheeky Icy Thang,STAYC,Metamorphic,150769
2,spotify:track:3pR7ZToCrzeNkfEf6n2rVR,B.A.S. (feat. Kyle Richh),"Megan Thee Stallion, Kyle Richh",MEGAN,206461
3,spotify:track:2QzOxrOZeP95cy5fEKzfwQ,Underground • アンダーグラウンド,f5ve,Underground • アンダーグラウンド,141782
4,spotify:track:7aOdnAhx1zWwtqAaXBOndI,Air Force One,ODD EYE CIRCLE (ARTMS),ODD EYE CIRCLE <Version Up>,163813
...,...,...,...,...,...
2266,spotify:track:3TVWi92dWPMVcKpXvX32UX,First Love,BTS,Wings,189795
2267,spotify:track:782ET7GKMVvNPBez4nsurW,FAKE LOVE (Rocking Vibe Mix),BTS,FAKE LOVE (Rocking Vibe Mix),238494
2268,spotify:track:7wvYmO5QscgIDDT0eNwbYX,Lost In Japan,Shawn Mendes,Lost In Japan,200133
2269,spotify:track:32fwe7MW5koort1Ua3YFx6,Terrapin,Clairo,Charm,180289


In [6]:
#Removing the track URI
track_info = df.drop(["track_uri"], axis=1)
track_info

,track_name,artist_name,album_name,track_duration
0,Bad Boy,Cascada,Everytime We Touch (Premium Edition),191843
1,Cheeky Icy Thang,STAYC,Metamorphic,150769
2,B.A.S. (feat. Kyle Richh),"Megan Thee Stallion, Kyle Richh",MEGAN,206461
3,Underground • アンダーグラウンド,f5ve,Underground • アンダーグラウンド,141782
4,Air Force One,ODD EYE CIRCLE (ARTMS),ODD EYE CIRCLE <Version Up>,163813
...,...,...,...,...
2266,First Love,BTS,Wings,189795
2267,FAKE LOVE (Rocking Vibe Mix),BTS,FAKE LOVE (Rocking Vibe Mix),238494
2268,Lost In Japan,Shawn Mendes,Lost In Japan,200133
2269,Terrapin,Clairo,Charm,180289


In [7]:
# Function to get all track IDs from a playlist
def get_playlist_track_ids(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    track_ids = [track['track']['id'] for track in tracks]
    return track_ids

# Function to get audio features for a list of track IDs
def get_audio_features(track_ids):
    features_list = []
    for i in range(0, len(track_ids), 100):  # Spotify API allows max 100 IDs per request
        audio_features = sp.audio_features(track_ids[i:i+100])
        features_list.extend(audio_features)
    return features_list

# Retrieving all track IDs from the playlist
track_ids = get_playlist_track_ids(playlist_id)

# Retrieving audio features for all tracks
audio_features = get_audio_features(track_ids)

# Converting the features to a DataFrame
audio_features = pd.DataFrame(audio_features)
audio_features


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.666,0.978,11,-5.118,0,0.0925,0.0149,0.0444,0.222,0.630,139.991,audio_features,1p5jgCAGFDBIBs6ZbWIMvA,spotify:track:1p5jgCAGFDBIBs6ZbWIMvA,https://api.spotify.com/v1/tracks/1p5jgCAGFDBI...,https://api.spotify.com/v1/audio-analysis/1p5j...,191843,4
1,0.757,0.843,2,-4.033,1,0.0475,0.0128,0.0000,0.117,0.673,131.011,audio_features,3BhyxulZ13uWZS6LqiYiCj,spotify:track:3BhyxulZ13uWZS6LqiYiCj,https://api.spotify.com/v1/tracks/3BhyxulZ13uW...,https://api.spotify.com/v1/audio-analysis/3Bhy...,150769,4
2,0.710,0.761,7,-3.424,1,0.2360,0.0473,0.0000,0.110,0.504,150.016,audio_features,3pR7ZToCrzeNkfEf6n2rVR,spotify:track:3pR7ZToCrzeNkfEf6n2rVR,https://api.spotify.com/v1/tracks/3pR7ZToCrzeN...,https://api.spotify.com/v1/audio-analysis/3pR7...,206461,4
3,0.633,0.892,9,-5.975,1,0.0687,0.0421,0.1140,0.142,0.517,176.920,audio_features,2QzOxrOZeP95cy5fEKzfwQ,spotify:track:2QzOxrOZeP95cy5fEKzfwQ,https://api.spotify.com/v1/tracks/2QzOxrOZeP95...,https://api.spotify.com/v1/audio-analysis/2QzO...,141782,4
4,0.742,0.938,6,-3.701,1,0.2140,0.0528,0.0000,0.132,0.516,137.986,audio_features,7aOdnAhx1zWwtqAaXBOndI,spotify:track:7aOdnAhx1zWwtqAaXBOndI,https://api.spotify.com/v1/tracks/7aOdnAhx1zWw...,https://api.spotify.com/v1/audio-analysis/7aOd...,163813,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,0.517,0.741,8,-5.997,0,0.3230,0.3620,0.0000,0.164,0.672,86.526,audio_features,3TVWi92dWPMVcKpXvX32UX,spotify:track:3TVWi92dWPMVcKpXvX32UX,https://api.spotify.com/v1/tracks/3TVWi92dWPMV...,https://api.spotify.com/v1/audio-analysis/3TVW...,189795,4
2267,0.463,0.804,2,-4.009,0,0.0391,0.0573,0.0000,0.432,0.352,154.913,audio_features,782ET7GKMVvNPBez4nsurW,spotify:track:782ET7GKMVvNPBez4nsurW,https://api.spotify.com/v1/tracks/782ET7GKMVvN...,https://api.spotify.com/v1/audio-analysis/782E...,238494,4
2268,0.726,0.708,7,-6.908,0,0.3560,0.3030,0.0000,0.348,0.417,105.084,audio_features,7wvYmO5QscgIDDT0eNwbYX,spotify:track:7wvYmO5QscgIDDT0eNwbYX,https://api.spotify.com/v1/tracks/7wvYmO5QscgI...,https://api.spotify.com/v1/audio-analysis/7wvY...,200133,4
2269,0.582,0.441,4,-7.394,1,0.0452,0.2310,0.7400,0.188,0.469,136.932,audio_features,32fwe7MW5koort1Ua3YFx6,spotify:track:32fwe7MW5koort1Ua3YFx6,https://api.spotify.com/v1/tracks/32fwe7MW5koo...,https://api.spotify.com/v1/audio-analysis/32fw...,180290,4


In [13]:
#Cleaning audio features DataFrame of unnecessary columns
cleaned_af = audio_features.drop(["id", "uri", "track_href", "analysis_url", "duration_ms", "time_signature", "type"], axis =1)
cleaned_af 

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.666,0.978,11,-5.118,0,0.0925,0.0149,0.0444,0.222,0.630,139.991
1,0.757,0.843,2,-4.033,1,0.0475,0.0128,0.0000,0.117,0.673,131.011
2,0.710,0.761,7,-3.424,1,0.2360,0.0473,0.0000,0.110,0.504,150.016
3,0.633,0.892,9,-5.975,1,0.0687,0.0421,0.1140,0.142,0.517,176.920
4,0.742,0.938,6,-3.701,1,0.2140,0.0528,0.0000,0.132,0.516,137.986
...,...,...,...,...,...,...,...,...,...,...,...
2266,0.517,0.741,8,-5.997,0,0.3230,0.3620,0.0000,0.164,0.672,86.526
2267,0.463,0.804,2,-4.009,0,0.0391,0.0573,0.0000,0.432,0.352,154.913
2268,0.726,0.708,7,-6.908,0,0.3560,0.3030,0.0000,0.348,0.417,105.084
2269,0.582,0.441,4,-7.394,1,0.0452,0.2310,0.7400,0.188,0.469,136.932


In [16]:
# Initializing list to collect track information
track_data = []
artist_genres = {}

# Retrieving all tracks from the playlist using pagination
offset = 0
while True:
    results = sp.playlist_tracks(playlist_id, limit=100, offset=offset)
    if not results['items']:
        break
    
    for item in results["items"]:
        track = item["track"]
        track_info = {
            "track_id": track["id"],
            "track_name": track["name"],
            "artist_names": ", ".join(artist["name"] for artist in track["artists"]),
            "album_name": track["album"]["name"],
            "track_duration": track["duration_ms"],
            "artist_ids": [artist["id"] for artist in track["artists"]]
        }
        track_data.append(track_info)
    
    offset += 100

# Collecting unique artist IDs
unique_artist_ids = list({artist_id for track in track_data for artist_id in track["artist_ids"]})

# Retrieving artist data and collect genres in batches
for i in range(0, len(unique_artist_ids), 50):
    batch_ids = unique_artist_ids[i:i+50]
    artists_data = sp.artists(batch_ids)
    for artist in artists_data["artists"]:
        artist_genres[artist["id"]] = artist["genres"]

# Adding genres to track data
for track in track_data:
    track_genres = set()
    for artist_id in track["artist_ids"]:
        track_genres.update(artist_genres.get(artist_id, []))
    track["genres"] = ", ".join(track_genres)

# Creating DataFrame from track_data list
track_genres = pd.DataFrame(track_data)

# Dropping the artist_ids column as it's no longer needed and displaying DataFrame
track_genres.drop(columns=["artist_ids"], inplace=True)
track_genres


,track_id,track_name,artist_names,album_name,track_duration,genres
0,1p5jgCAGFDBIBs6ZbWIMvA,Bad Boy,Cascada,Everytime We Touch (Premium Edition),191843,"melbourne bounce international, german techno,..."
1,3BhyxulZ13uWZS6LqiYiCj,Cheeky Icy Thang,STAYC,Metamorphic,150769,"k-pop girl group, k-pop"
2,3pR7ZToCrzeNkfEf6n2rVR,B.A.S. (feat. Kyle Richh),"Megan Thee Stallion, Kyle Richh",MEGAN,206461,"pop, houston rap, trap queen, rap, new york dr..."
3,2QzOxrOZeP95cy5fEKzfwQ,Underground • アンダーグラウンド,f5ve,Underground • アンダーグラウンド,141782,
4,7aOdnAhx1zWwtqAaXBOndI,Air Force One,ODD EYE CIRCLE (ARTMS),ODD EYE CIRCLE <Version Up>,163813,
...,...,...,...,...,...,...
2266,3TVWi92dWPMVcKpXvX32UX,First Love,BTS,Wings,189795,"k-pop boy group, k-pop, pop"
2267,782ET7GKMVvNPBez4nsurW,FAKE LOVE (Rocking Vibe Mix),BTS,FAKE LOVE (Rocking Vibe Mix),238494,"k-pop boy group, k-pop, pop"
2268,7wvYmO5QscgIDDT0eNwbYX,Lost In Japan,Shawn Mendes,Lost In Japan,200133,"canadian pop, viral pop, pop"
2269,32fwe7MW5koort1Ua3YFx6,Terrapin,Clairo,Charm,180289,"indie pop, pov: indie, bedroom pop"


In [17]:
#Combinign the cleaned audio features DataFrame with the track genres DataFrame
full_playlist = pd.concat([track_genres, cleaned_af], axis=1)
full_playlist

,track_id,track_name,artist_names,album_name,track_duration,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1p5jgCAGFDBIBs6ZbWIMvA,Bad Boy,Cascada,Everytime We Touch (Premium Edition),191843,"melbourne bounce international, german techno,...",0.666,0.978,11,-5.118,0,0.0925,0.0149,0.0444,0.222,0.630,139.991
1,3BhyxulZ13uWZS6LqiYiCj,Cheeky Icy Thang,STAYC,Metamorphic,150769,"k-pop girl group, k-pop",0.757,0.843,2,-4.033,1,0.0475,0.0128,0.0000,0.117,0.673,131.011
2,3pR7ZToCrzeNkfEf6n2rVR,B.A.S. (feat. Kyle Richh),"Megan Thee Stallion, Kyle Richh",MEGAN,206461,"pop, houston rap, trap queen, rap, new york dr...",0.710,0.761,7,-3.424,1,0.2360,0.0473,0.0000,0.110,0.504,150.016
3,2QzOxrOZeP95cy5fEKzfwQ,Underground • アンダーグラウンド,f5ve,Underground • アンダーグラウンド,141782,,0.633,0.892,9,-5.975,1,0.0687,0.0421,0.1140,0.142,0.517,176.920
4,7aOdnAhx1zWwtqAaXBOndI,Air Force One,ODD EYE CIRCLE (ARTMS),ODD EYE CIRCLE <Version Up>,163813,,0.742,0.938,6,-3.701,1,0.2140,0.0528,0.0000,0.132,0.516,137.986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,3TVWi92dWPMVcKpXvX32UX,First Love,BTS,Wings,189795,"k-pop boy group, k-pop, pop",0.517,0.741,8,-5.997,0,0.3230,0.3620,0.0000,0.164,0.672,86.526
2267,782ET7GKMVvNPBez4nsurW,FAKE LOVE (Rocking Vibe Mix),BTS,FAKE LOVE (Rocking Vibe Mix),238494,"k-pop boy group, k-pop, pop",0.463,0.804,2,-4.009,0,0.0391,0.0573,0.0000,0.432,0.352,154.913
2268,7wvYmO5QscgIDDT0eNwbYX,Lost In Japan,Shawn Mendes,Lost In Japan,200133,"canadian pop, viral pop, pop",0.726,0.708,7,-6.908,0,0.3560,0.3030,0.0000,0.348,0.417,105.084
2269,32fwe7MW5koort1Ua3YFx6,Terrapin,Clairo,Charm,180289,"indie pop, pov: indie, bedroom pop",0.582,0.441,4,-7.394,1,0.0452,0.2310,0.7400,0.188,0.469,136.932


In [18]:
#More cleaning
Liked_Songs = full_playlist.drop(["track_id", "album_name"], axis=1)
Liked_Songs

,track_name,artist_names,track_duration,genres,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,Bad Boy,Cascada,191843,"melbourne bounce international, german techno,...",0.666,0.978,11,-5.118,0,0.0925,0.0149,0.0444,0.222,0.630,139.991
1,Cheeky Icy Thang,STAYC,150769,"k-pop girl group, k-pop",0.757,0.843,2,-4.033,1,0.0475,0.0128,0.0000,0.117,0.673,131.011
2,B.A.S. (feat. Kyle Richh),"Megan Thee Stallion, Kyle Richh",206461,"pop, houston rap, trap queen, rap, new york dr...",0.710,0.761,7,-3.424,1,0.2360,0.0473,0.0000,0.110,0.504,150.016
3,Underground • アンダーグラウンド,f5ve,141782,,0.633,0.892,9,-5.975,1,0.0687,0.0421,0.1140,0.142,0.517,176.920
4,Air Force One,ODD EYE CIRCLE (ARTMS),163813,,0.742,0.938,6,-3.701,1,0.2140,0.0528,0.0000,0.132,0.516,137.986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266,First Love,BTS,189795,"k-pop boy group, k-pop, pop",0.517,0.741,8,-5.997,0,0.3230,0.3620,0.0000,0.164,0.672,86.526
2267,FAKE LOVE (Rocking Vibe Mix),BTS,238494,"k-pop boy group, k-pop, pop",0.463,0.804,2,-4.009,0,0.0391,0.0573,0.0000,0.432,0.352,154.913
2268,Lost In Japan,Shawn Mendes,200133,"canadian pop, viral pop, pop",0.726,0.708,7,-6.908,0,0.3560,0.3030,0.0000,0.348,0.417,105.084
2269,Terrapin,Clairo,180289,"indie pop, pov: indie, bedroom pop",0.582,0.441,4,-7.394,1,0.0452,0.2310,0.7400,0.188,0.469,136.932


In [ ]:
#Converting the DataFrame to a csv file
Liked_Songs.to_csv("My_Songs.csv", index=False)